In [ ]:
!pip install basedosdados
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.0
    Uninstalling pyarrow-6.0.0:
      Successfully uninstalled pyarrow-6.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
basedosdados 1.6.11 requires pyarrow==6.0.0, but you have pyarrow 15.0.0 which is incompatible.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompat

In [ ]:
import pandas as pd
import basedosdados as bd
import streamlit as st
import altair as alt
import plotly.express as px
import ast

In [ ]:
df_chamados = bd.read_sql("SELECT * FROM `datario.administracao_servicos_publicos.chamado_1746`", billing_project_id="seismic-harmony-413819")

Downloading: 100%|██████████| 10742832/10742832 [58:49<00:00, 3043.94rows/s]


#Variáveis Globais

In [ ]:
# Variaveis a serem utilizadas nas queries
data = pd.to_datetime("2023-04-01")
subtipo = "Perturbação do sossego"
range = (pd.to_datetime("2022-01-01"),pd.to_datetime("2023-12-31"))

df_chamados_filtrado = df_chamados[df_chamados['data_inicio'].dt.date == data]
df_bairro = bd.read_sql("SELECT * FROM `datario.dados_mestres.bairro`", billing_project_id="seismic-harmony-413819")
df_eventos = bd.read_sql("SELECT * FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`", billing_project_id="seismic-harmony-413819")

chamados_pertubação_sossego = df_chamados[df_chamados['subtipo'] == 'Perturbação do sossego']
chamados_com_eventos = pd.merge(chamados_pertubação_sossego, df_eventos, how='cross')
chamados_com_eventos = chamados_com_eventos[(chamados_com_eventos['data_inicio'].dt.date >= chamados_com_eventos['data_inicial']) & (chamados_com_eventos['data_inicio'].dt.date <= chamados_com_eventos['data_final'])]
chamados_com_eventos = chamados_com_eventos[['evento', 'data_inicio']]


Downloading: 100%|██████████| 4/4 [00:00<00:00, 20.23rows/s]


#Resposta 1

In [ ]:
numero_chamados = df_chamados_filtrado.shape[0]
numero_chamados

73

#Resposta 2

In [ ]:
df_2 = df_chamados_filtrado
df_2 = df_2.groupby('tipo').size().reset_index(name='total_chamados')
df_2_sorted = df_2.sort_values('total_chamados', ascending=False)
tipo = df_2_sorted['tipo'].iloc[0]
tipo

'Poluição sonora'

#Resposta 3

In [ ]:
df = pd.merge(df_chamados_filtrado, df_bairro, on='id_bairro')
df.columns
df = df.groupby('nome').size().reset_index(name='num_chamados')
df = df.sort_values('num_chamados', ascending=False)
tres_bairros_com_mais_chamados = df.head(3)
tres_bairros_com_mais_chamados

,nome,num_chamados
10,Engenho de Dentro,8
3,Campo Grande,6
20,Leblon,6


#Resposta 4

In [ ]:
df = pd.merge(df_chamados_filtrado, df_bairro, on='id_bairro')
df = df.groupby('subprefeitura').size().reset_index(name='num_chamados')
df = df.sort_values('num_chamados', ascending=False)
subprefeitura_com_mais_chamados = df['subprefeitura'].iloc[0]
subprefeitura_com_mais_chamados

'Zona Norte'

#Resposta 5

Existe chamados sem bairro pois ao realizar esse tipo de reclamação não há a necessidade de informar o local. Isso pode ser confirmado no site oficial, onde os campos de localidade para esse tipo de reclamação são opcionais.

In [ ]:
chamados_sem_bairro = df_chamados_filtrado[df_chamados_filtrado['id_bairro'].isnull()]
chamados_sem_bairro

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,geometry
2427371,18516246,2023-04-01 00:55:38,2023-04-01 00:55:38,None,None,None,<NA>,1706,TR/SUBOP/CFT - Coordenadoria de Fiscalização e...,SMTR - Secretaria Municipal de Transportes,...,<NA>,D,F,No prazo,Encerrado,Atendido parcialmente,None,0,2023-04-01,None


#Resposta 6

In [ ]:
chamados_com_eventos = pd.merge(chamados_pertubação_sossego, df_eventos, how='cross')

In [ ]:
chamados = chamados_pertubação_sossego[(chamados_pertubação_sossego['data_inicio'].dt.date >= range[0]) & (chamados_pertubação_sossego['data_inicio'].dt.date <= range[1])]
numero_de_chamados = chamados.shape[0]
numero_de_chamados

42408

#Resposta 7

In [ ]:
chamados_com_eventos = chamados_com_eventos[(chamados_com_eventos['data_inicio'].dt.date >= chamados_com_eventos['data_inicial']) & (chamados_com_eventos['data_inicio'].dt.date <= chamados_com_eventos['data_final'])]
chamados_com_eventos

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,tipo_situacao,justificativa_status,reclamacoes,data_particao,geometry,ano,data_inicial,data_final,evento,taxa_ocupacao
2265,18078353,2022-12-30 22:51:00,NaT,106,3,275487,10,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Andamento,None,0,2022-12-01,None,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2826,17658272,2022-09-02 01:24:32,2022-09-09 08:08:08,24,2,68882,403,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Não atendido,None,0,2022-09-01,POINT(-43.1864773 -22.9707463),02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
2968,18334310,2023-02-21 17:38:41,2023-02-25 19:09:59,128,4,204339,890,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Não atendido,None,0,2023-02-01,POINT(-43.3078638 -23.0147498),18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
2972,18333025,2023-02-20 20:27:15,2023-02-28 10:27:40,119,4,115683,171,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Não atendido,None,0,2023-02-01,POINT(-43.38441 -22.9456099),18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
3094,17658256,2022-09-02 00:36:54,2022-09-09 07:59:39,35,2,71415,164,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Não atendido,None,0,2022-09-01,POINT(-43.234505 -22.918669),02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361015,17685362,2022-09-11 19:01:51,2022-09-19 10:19:45,89,3,19646,189,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Não atendido,None,0,2022-09-01,POINT(-43.364969 -22.8679822),08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451
361052,18331953,2023-02-20 11:44:50,2023-02-24 07:25:04,26,2,65888,1166,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Não atendido,None,0,2023-02-01,POINT(-43.2228142 -22.9843111),18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
361159,17676859,2022-09-08 14:09:44,2022-09-16 08:05:30,24,2,66514,355,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Não atendido,None,1,2022-09-01,POINT(-43.18449 -22.9679422),08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451
361163,17684695,2022-09-11 11:01:04,2022-09-19 09:28:16,29,2,80986,79,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Não atendido,None,0,2022-09-01,POINT(-43.2260473 -22.9732124),08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451


#Resposta 8

In [ ]:
chamados_por_evento = chamados_com_eventos.groupby('evento').size()
chamados_por_evento = pd.DataFrame({'evento':chamados_por_evento.index, 'num_chamados':chamados_por_evento.values})
chamados_por_evento

,evento,num_chamados
0,Carnaval,241
1,Reveillon,137
2,Rock in Rio,834


#Resposta 9

In [ ]:
chamados_com_eventos['data_chamada'] = pd.to_datetime(chamados_com_eventos['data_inicio']).dt.date
num_chamados_por_dia = chamados_com_eventos.groupby(['data_chamada', 'evento']).size()
media_chamados_por_dia = num_chamados_por_dia.groupby('evento').mean()
media_chamados_por_dia.sort_values().index[-1]

'Rock in Rio'

#Resposta 10

In [ ]:
# ## Consulta 1: Média de chamados por dia em 2 anos

chamados_2_anos = chamados_pertubação_sossego[(chamados_pertubação_sossego['data_inicio'].dt.date >= range[0]) & (chamados_pertubação_sossego['data_inicio'].dt.date <= range[1])]
chamados_2_anos['data_chamada'] = pd.to_datetime(chamados_2_anos['data_inicio']).dt.date
num_chamados_por_dia_2_anos = chamados_2_anos.groupby('data_chamada').size()
media_chamados_por_dia_2_anos = num_chamados_por_dia_2_anos.mean()
print(f"Média de chamados por dia em 2 anos: {media_chamados_por_dia_2_anos}")

# ## Consulta 2: Média de chamados por dia para eventos

chamados_com_eventos = chamados_com_eventos[(chamados_com_eventos['data_inicio'].dt.date >= chamados_com_eventos['data_inicial']) & (chamados_com_eventos['data_inicio'].dt.date <= chamados_com_eventos['data_final'])]
chamados_com_eventos['data_chamada'] = pd.to_datetime(chamados_com_eventos['data_inicio']).dt.date
num_chamados_por_dia = chamados_com_eventos.groupby(['data_chamada', 'evento']).size()
media_chamados_por_dia = num_chamados_por_dia.mean()
print(f"Média de chamados por dia em eventos:")
print(media_chamados_por_dia)

Média de chamados por dia em 2 anos: 63.20119225037258
Média de chamados por dia em eventos:
86.57142857142857
